# [NTDS'18] milestone 2: network models
[ntds'18]: https://github.com/mdeff/ntds_2018

[Hermina Petric Maretic](https://people.epfl.ch/hermina.petricmaretic), [EPFL LTS4](https://lts4.epfl.ch)

## Students

* Team: `28`
* Students: `Guillain, Léonore Valentine; Pase, Francesco; Rusu, Cosmin-Ionut; Zhuang, Ying`
* Dataset: `Flight Routes`

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to two sentences.
* Code has to be clean.
* In the first part, you cannot import any other library than we imported. In the second part, you are allowed to import any library you want.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective

The purpose of this milestone is to explore various random network models, analyse their properties and compare them to your network. In the first part of the milestone you will implement two random graph models and try to fit them to your network. In this part you are not allowed to use any additional package. In the second part of the milestone you will choose a third random graph model that you think shares some properties with your network. You will be allowed to use additional packages to construct this network, but you must explain your network choice. Finally, make your code as clean as possible, and keep your textual answers short.

## Part 0

Import the adjacency matrix of your graph that you constructed in milestone 1, as well as the number of nodes and edges of your network.

In [ ]:
# Import these earlier for reproducibility
import numpy as np
import pandas as pd

adjacency = np.load('adjacency.npy') # the adjacency matrix
n_nodes =  adjacency.shape[0] # the number of nodes in the network
n_edges =  adjacency.sum() # the number of edges in the network

## Part 1

**For the computation of this part of the milestone you are only allowed to use the packages that have been imported in the cell below.**

In [ ]:
%matplotlib inline

import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

### Question 1

Create a function that constructs an Erdős–Rényi graph.

In [ ]:
def erdos_renyi(n, p, seed=None):
    """Create an instance from the Erdos-Renyi graph model.
    
    Parameters
    ----------
    n: int
        Size of the graph.
    p: float
        Edge probability. A number between 0 and 1.
    seed: int (optional)
        Seed for the random number generator. To get reproducible results.
    
    Returns
    -------
    adjacency
        The adjacency matrix of a graph.
    """
    
    # Your code here.
    
    np.random.seed(seed)
    adjacency = np.zeros((n,n))
    for i in range(n):
        adjacency[i, i:] = np.random.binomial(1, p, n-i)
        adjacency[i:, i] = adjacency[i, i:]
        
    
    
    return adjacency

In [ ]:
er = erdos_renyi(5, 0.6, 9765)
plt.spy(er)
plt.title('Erdos-Renyi (5, 0.6)')

In [ ]:
er = erdos_renyi(10, 0.4, 7648)
plt.spy(er)
plt.title('Erdos-Renyi (10, 0.4)')

### Question 2

Use the function to create a random Erdos-Renyi graph. Choose the parameters such that number of nodes is the same as in your graph, and the number of edges similar. You don't need to set the random seed. Comment on your choice of parameters.

In [ ]:
# Your code here.
edges = np.where(adjacency > 0, 1, 0).sum()/2
edges

In [ ]:
p = 2*edges/(n_nodes*(n_nodes-1))
p

In [ ]:
erdos_renyi(n_nodes, p).sum()/2

In [ ]:
er_graph = erdos_renyi(n_nodes, p)
plt.spy(er_graph)

**Your answer here.**

### Question 3

Create a function that constructs a Barabási-Albert graph.

In [ ]:
def barabasi_albert(n, m, seed=None):
    """Create an instance from the Barabasi-Albert graph model.
    
    Parameters
    ----------
    n: int
        Size of the graph.
    m: int
        Number of edges to attach from a new node to existing nodes.
    seed: int (optional)
        Seed for the random number generator. To get reproducible results.
    
    Returns
    -------
    adjacency
        The adjacency matrix of a graph.
    """
    
    # Your code here.
    np.random.seed(seed)
    # Initial connected netwoprk
    m_0 = np.random.randint(m, n)
    t = n - m_0
    adjacency = np.zeros((n,n))
    
    #initialize
    for i in range(m_0):
        adjacency[i, :m_0-i] = np.random.randint(0, 1, m_0-i)
        # while i is not connected to at least one element from the other nodes
        while  np.where(adjacency[i, :m_0-i] == 0, 0, 1).sum() == 0:
            adjacency[i, np.random.randint(0, m_0)] = 1

        adjacency[:m_0, i] = adjacency[i, :m_0]
    
    #iterate:
    for i in range(t):
        #we have i+m_0 nodes already in the network
        degrees = adjacency[:m_0 + i].sum(axis = 1)
        total = degrees.sum()
        new_links = np.random.choice(np.arange(m_0 + i),
                                    size=m,
                                    replace=False,
                                    p = degrees / total)
        adjacency[m_0+i, new_links] = 1.0
        adjacency[:m_0+i, m_0+i] = adjacency[m_0+i, :m_0+i]
    
    return adjacency

In [ ]:
#bad see, is all zero
ba = barabasi_albert(5, 1, 908)
plt.spy(ba)
plt.title('Barabasi-Albert (5, 1)')

In [ ]:
ba = barabasi_albert(10, 2, 8708)
plt.spy(ba)
plt.title('Barabasi-Albert (10, 2)')

### Question 4

Use the function to create a random Barabási-Albert graph. Choose the parameters such that number of nodes is the same as in your graph, and the number of edges similar. You don't need to set the random seed. Comment on your choice of parameters.

In [ ]:
avg_degree = adjacency.sum(axis = 1).mean()
avg_degree

In [ ]:
# Your code here.
ba_graph = barabasi_albert(n_nodes, int(avg_degree))
plt.spy(ba_graph)

**Your answer here**

### Question 5

Compare the number of edges in all three networks (your real network, the Erdős–Rényi network, and the Barabási-Albert netowk).

In [ ]:
# Your code here.
print(f"Number of edges in the real network: {n_edges}")
print(f"Number of edges in the Erdős–Rényi network: {er_graph.sum() / 2}")
print(f"Number of edges in the Barabási-Albert network: {ba_graph.sum() / 2}")

### Question 6

Implement a function that computes the [Kullback–Leibler (KL) divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) between two probability distributions.
We'll use it to compare the degree distributions of networks.

In [ ]:
def kl_divergence(p, q):
    """Compute the KL divergence between probability distributions of degrees of two networks.
    
    Parameters
    ----------
    p: np.array
        Probability distribution of degrees of the 1st graph.
    q: np.array
        Probability distribution of degrees of the 2nd graph.
    
    Returns
    -------
    kl
        The KL divergence between the two distributions.
    """
    
    # Your code here.
    kl = (np.log(p / q) * p).sum()
    
    return kl

In [ ]:
p_test = np.array([0.2, 0.2, 0.2, 0.4])
q_test = np.array([0.3, 0.3, 0.1, 0.3])
kl_divergence(p_test, q_test)

### Question 7

Compare the degree distribution of your network to each of the two synthetic ones, in terms of KL divergence. **Hint:** Make sure you normalise your degree distributions to make them valid probability distributions.

In [ ]:
# Your code here.
degree_network = adjacency.sum(axis = 1) / adjacency.sum()
degree_er_network = er_graph.sum(axis = 1) / er_graph.sum()
degree_ba_network = ba_graph.sum(axis = 1) / ba_graph.sum()

assert(np.abs(1 - degree_network.sum()) < 0.0001)
assert(np.abs(1 - degree_er_network.sum()) < 0.001)
assert(np.abs(1 - degree_ba_network.sum()) < 0.001)

print(f"KL Divergence between the original network and the Erdős–Rényi network: {kl_divergence(degree_network, degree_er_network)}")
print(f"KL Divergence between the original network and the Barabási-Albert network: {kl_divergence(degree_network, degree_ba_network)}")


### Question 8

Plot the degree distribution historgrams for all three networks. Are they consistent with the KL divergence results? Explain.

In [ ]:
# Your code here.
_ = plt.hist(adjacency.sum(axis = 1))

In [ ]:
_ = plt.hist(er_graph.sum(axis = 1))

In [ ]:
_ = plt.hist(ba_graph.sum(axis = 1))

**Your answer here.**

### Question 9

Imagine you got equal degree distributions. Would that guarantee you got the same graph? Explain.

# TODO(someone): Proof read my (cosmin) answer:
If we have the same degree distrubtion does not mean we have the same graph. If two graphs have same degree sequence, that does not mean that the graphs are isomorphic, hence they cannot be the same. To proof this, we will find a counter example. Consider two graphs with the same degree distribution: (2, 2, 2, 1, 1):

```
* 1 -- 2 -- 3 -- 4 -- 5
* 1 -- 2    3 -- 4
             \ /
              5
```

## Part 2

**You are allowed to use any additional library here (e.g., NetworkX, PyGSP, etc.).** Be careful not to include something here and use it in part 1!

In [ ]:
import networkx as nx

### Question 10

Choose a random network model that fits you network well. Explain your choice. 

**Hint:** Check lecture notes for different network models and their properties. Your choice should be made based on at least one property you'd expect to be similar.

**Your answer here.**

### Question 11

Explain (in short) how the chosen model works.

**Your answer here.**

### Question 12

Create a random graph from that model, such that the number of nodes is the same as in your graph.

In [ ]:
# Your code here.

### Question 13

Check the properties you expected to be similar, and compare to your network.

In [ ]:
# Your code here.

Are the results what you expected? Explain.

**Your answer here.**